In [46]:
import numpy as np
from scipy.spatial import distance

def hexdec(s):
    return bytes.fromhex(s)

def fixed_xor(b1, b2):
    a = np.frombuffer(b1, 'u1')
    b = np.frombuffer(b2, 'u1')
    return (a ^ b).tobytes()

ascii_hist = lambda s: np.histogram(s, bins=range(128), density=True)[0]

with open('t8.shakespeare.txt', 'r') as f:
    shks = np.frombuffer(f.read().encode("ASCII"), 'u1')

shks_hist = ascii_hist(shks)

score = lambda buf: distance.jensenshannon(shks_hist, ascii_hist(np.frombuffer(buf, 'u1')))

i2b = lambda i: bytes([i])

def break_sb_xor_key(cipher):
    evaluation = np.array([score(fixed_xor(cipher, i2b(i) * len(cipher))) for i in range(128)])
    return i2b(np.argmin(evaluation))

def decipher_sb_xor_key(cipher):
    key = break_sb_xor_key(cipher)
    return fixed_xor(key, cipher)


In [6]:
with open("4.txt", "r") as f:
    ciphers = f.read().splitlines()

In [48]:
cipher_scores = np.array([ score(decipher_sb_xor_key(hexdec(cipher))) for cipher in ciphers ])

In [52]:
decipher_sb_xor_key(hexdec(ciphers[np.argmin(cipher_scores)]))

b'Now that the party is jumping\n'